In [1]:
from surprise import Dataset
from surprise import Dataset, Reader
from surprise import Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV
import pickle

In [1]:
import pickle
import pandas as pd

In [2]:
import gzip
import tracemalloc


In [2]:
with open('modelo_entrenado.pkl', 'rb') as file:
    model = pickle.load(file)

In [20]:
opinion = pd.read_csv(r'data_set_limpio/reviews_preparado.csv.gz')

In [21]:
df = pd.read_csv(r'data_set_limpio/games_dummies.csv.gz')

In [22]:
entrenar = pd.read_csv(r'data_set_limpio\\modelo.csv.gz')

In [23]:
prueba1 = pd.merge(opinion[['item_id','user','recommend','opinion','helpful']], df[['id','app_name']], left_on='item_id', right_on='id')

prueba1['recommend'] = prueba1['recommend'].replace({True: 1, False: 0})

prueba1.loc[prueba1['helpful'] == 'No ratings yet', 'helpful'] = 0

prueba1.loc[prueba1['helpful']  != 0 , 'helpful'] = 1

In [24]:
prueba1[5000:6000]

,item_id,user,recommend,opinion,helpful,id,app_name
5000,730,TheBreakfastCrew,1,2,0,730,Counter-Strike: Global Offensive
5001,730,76561198083640655,1,2,0,730,Counter-Strike: Global Offensive
5002,730,76561198083674299,1,0,1,730,Counter-Strike: Global Offensive
5003,730,jarlow,1,2,1,730,Counter-Strike: Global Offensive
5004,730,inorisanbaka,0,1,1,730,Counter-Strike: Global Offensive
...,...,...,...,...,...,...,...
5995,550,76561198075577878,1,1,0,550,Left 4 Dead 2
5996,550,MarsupialesLocos,1,0,1,550,Left 4 Dead 2
5997,550,76561198091526444,1,0,1,550,Left 4 Dead 2
5998,550,76561198056984812,1,0,0,550,Left 4 Dead 2


In [25]:
prueba1.loc[prueba1['user'] == 'inorisanbaka']

,item_id,user,recommend,opinion,helpful,id,app_name
5004,730,inorisanbaka,0,1,1,730,Counter-Strike: Global Offensive


In [393]:
for index, row in prueba1.iterrows():
    if (row[2] == 1) and (row[3] == 2) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 4
    elif (row[2] == 1) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 0) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 2) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 1) and (row[3] == 2) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 3
    elif (row[2] == 1) and (row[3] == 1) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 2
    elif (row[2] == 1) and (row[3] == 0) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 0) and (row[4] == 0):
        prueba1.at[index, 'rating'] = 0
    elif (row[2] == 0) and (row[3] == 0) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 1
    elif (row[2] == 0) and (row[3] == 1) and (row[4] == 1):
        prueba1.at[index, 'rating'] = 2
    else:
        prueba1.at[index, 'rating'] = 2
        

C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[2] == 1) and (row[3] == 2) and (row[4] == 1):
C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[2] == 1) and (row[3] == 1) and (row[4] == 1):
C:\Users\jonat\AppData\Local\Temp\ipykernel_25504\1727096209.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.

In [413]:
entrenar = prueba1[['user','app_name','rating']]

In [438]:
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import train_test_split
import pandas as pd

# Supongamos que tienes un DataFrame 'df' con tus datos
# Columnas: 'user', 'app_name', 'rating'

# Crear un objeto Reader
reader = Reader(rating_scale=(0, 4))  # Adaptar rating_scale según tu escala de calificación

# Cargar los datos con el lector
data = Dataset.load_from_df(entrenar[['user', 'app_name', 'rating']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)  # Puedes ajustar el tamaño de prueba

# Inicializar el modelo SVD++
model = SVDpp(n_factors = 50, n_epochs = 25, lr_all= 0.005, reg_all= 0.04)  # Estos son valores de ejemplo

# Entrenar el modelo
model.fit(trainset)

# Realizar predicciones en el conjunto de prueba
predictions = model.test(testset)

# Evaluar las predicciones (por ejemplo, calcular el RMSE)
from surprise import accuracy
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


RMSE: 1.0196
RMSE: 1.0196406487549206


In [4]:
def recomendaciones_usuario(usuario):
        
    juegos_valorados = entrenar[entrenar['user'] == usuario]['app_name'].unique()

    todos_los_juegos = entrenar['app_name'].unique()

    juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

    predicciones = [model.predict(usuario, juego) for juego in juegos_no_valorados]
    
    recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5] 

    diccionario = {}
    
    for recomendacion in recomendaciones:
        
        diccionario[recomendacion.iid] = recomendacion.est
        
    return diccionario

In [17]:
testeo1 = recomendaciones_usuario('death-hunter')
testeo1

{'LEGO® Star Wars™ - The Complete Saga': 3.2881163193343927,
 'Batman - The Telltale Series': 3.286170343197686,
 'STAR WARS™ Knights of the Old Republic™ II - The Sith Lords™': 3.2780873324782243,
 'Resident Evil 6 / Biohazard 6': 3.276265852996828,
 'Bit Blaster XL': 3.2573728017414423}

In [8]:
usuarios = entrenar['user'].unique()

In [9]:
diccionario = {}

for x in usuarios[:30]:
    
    diccionario[x] = recomendaciones_usuario(x)

diccionario

{'76561197970982479': {'Magicite': 3.3962200315633413,
  'Bit Blaster XL': 3.360933976578513,
  'The Elder Scrolls® Online: Tamriel Unlimited™': 3.299575407543332,
  'Tales from the Borderlands': 3.2755591256282655,
  "Sid Meier's Civilization V: Brave New World": 3.255833395272257},
 'death-hunter': {'LEGO® Star Wars™ - The Complete Saga': 3.2881163193343927,
  'Batman - The Telltale Series': 3.286170343197686,
  'STAR WARS™ Knights of the Old Republic™ II - The Sith Lords™': 3.2780873324782243,
  'Resident Evil 6 / Biohazard 6': 3.276265852996828,
  'Bit Blaster XL': 3.2573728017414423},
 'DJKamBer': {'Batman - The Telltale Series': 3.473209369870282,
  'Bit Blaster XL': 3.4650470304535927,
  'Dragon Age: Origins': 3.4268998197484626,
  'Resident Evil 6 / Biohazard 6': 3.4170923393908583,
  'Life is Strange - Episode 1': 3.415780348342124},
 'diego9031': {'Tales from the Borderlands': 2.5916565454572758,
  'Bit Blaster XL': 2.5548115024582754,
  'Batman - The Telltale Series': 2.5380

In [18]:
entrenar.loc[entrenar['user'] == 'MeloncraftLP']

,item_id,user,recommend,opinion,id,app_name,rating
34541,265630,MeloncraftLP,0,2,265630,Fistful of Frags,2.0
49592,220090,MeloncraftLP,1,0,220090,The Journey Down: Chapter One,1.0
49593,262850,MeloncraftLP,1,0,262850,The Journey Down: Chapter Two,1.0


In [21]:
prueba1.to_csv('data_set_limpio/modelo.csv',encoding='UTF-8',index=False)

In [22]:
with open('data_set_limpio/modelo.csv', 'rb') as f_in, gzip.open('data_set_limpio/modelo.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)

In [425]:
with open('modelo_entrenado.pkl', 'wb') as file:
    pickle.dump(model, file)

Mejores parámetros encontrados: {'n_factors': 50, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.04}

In [361]:
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import GridSearchCV

# Supongamos que 'entrenar' es tu DataFrame con las columnas: 'user', 'app_name', 'rating'

# Crear un objeto Reader
reader = Reader(rating_scale=(0, 4))  # Ajustar según la escala de calificación

# Cargar los datos con el lector
data = Dataset.load_from_df(entrenar[['user', 'app_name', 'rating']], reader)

# Definir el espacio de parámetros a explorar
'''param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'init_mean': [0, 0.5],
    'init_std_dev': [0, 0.5],
    'random_state': [0, 42],  # Cambia los valores de la semilla si es necesario
    'verbose': [True, False]
}'''
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 25],  # Reducir el rango de epochs
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'random_state': [0, 42]
}

# Inicializar el modelo SVD++
model = SVDpp()

# Realizar la búsqueda de cuadrícula
grid_search = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3)

# Ajustar la búsqueda de cuadrícula con los datos
grid_search.fit(data)

# Obtener los mejores resultados
best_params = grid_search.best_params['rmse']
print("Mejores parámetros encontrados:", best_params)

#Mejores parámetros encontrados: {'n_factors': 50, 'n_epochs': 25, 'lr_all': 0.005, 'reg_all': 0.04, 'random_state': 42}

Mejores parámetros encontrados: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.04, 'random_state': 0}


In [362]:
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import GridSearchCV

# Supongamos que 'entrenar' es tu DataFrame con las columnas: 'user', 'app_name', 'rating'

# Crear un objeto Reader
reader = Reader(rating_scale=(0, 4))  # Ajustar según la escala de calificación

# Cargar los datos con el lector
data = Dataset.load_from_df(entrenar[['user', 'app_name', 'rating']], reader)

# Definir el espacio de parámetros a explorar
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'init_mean': [0, 0.5],
    'init_std_dev': [0, 0.5],
    'random_state': [0, 42],  # Cambia los valores de la semilla si es necesario
    'verbose': [True, False]
}
'''param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 25],  # Reducir el rango de epochs
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.04],
    'random_state': [0, 42]
}'''

# Inicializar el modelo SVD++
model = SVDpp()

# Realizar la búsqueda de cuadrícula
grid_search = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3)

# Ajustar la búsqueda de cuadrícula con los datos
grid_search.fit(data)

# Obtener los mejores resultados
best_params = grid_search.best_params['rmse']
print("Mejores parámetros encontrados:", best_params)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
